## 常见类型

### PromptTemplate

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="请用 GPT-4o 的视角写一段关于 {topic} 的介绍，控制在50字以内。"
)

prompt_format = prompt.format(topic="Python")

print(prompt_format)

请用 GPT-4o 的视角写一段关于 Python 的介绍，控制在50字以内。


### ChatPromptTemplate

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个精通中文的 GPT-4o 助手，专门讲解技术概念。"),
    ("user", "请用 GPT-4o 的能力讲解 {concept}，要求简洁明了。")
])

# 正确的格式化方式，需要提供 concept 参数
prompt_format = chat_template.format(concept="LangChain")

print(prompt_format)

System: 你是一个精通中文的 GPT-4o 助手，专门讲解技术概念。
Human: 请用 GPT-4o 的能力讲解 LangChain，要求简洁明了。


## 常用功能

### 多变量支持

In [4]:
multi_prompt = PromptTemplate(
    input_variables=["concept", "level", "language"],
    template="请用 {language} 向 {level} 的学习者解释 {concept}，要求简洁明了。"
)

prompt_format = multi_prompt.format(concept="Python", level="初中生", language="中文")

print(prompt_format)

请用 中文 向 初中生 的学习者解释 Python，要求简洁明了。


### 链式调用 (LCEL语法)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate(
    input_variables=["topic"],
    template="请用 GPT-4o 的视角写一段关于 {topic} 的介绍，控制在50字以内。"
)

llm = init_chat_model("gpt-4o-mini")

chain = prompt | llm | StrOutputParser()

result = chain.invoke({"topic": "LangChain"})

print(result)

LangChain 是一个强大的框架，旨在简化构建基于语言模型的应用程序，通过模块化的组件和丰富的工具库，助力开发者快速创新和实现智能对话、文本生成等功能。


### 模板验证

#### 参数缺失校验

In [7]:
from langchain_core.prompts import PromptTemplate

# 创建模板
prompt = PromptTemplate(
    input_variables=["topic"],
    template="介绍 {topic}"
)

# ❌ 缺少变量 - 会报错
try:
    result = prompt.format()  # 没有提供 topic
    print(result)
except KeyError as e:
    print(f"错误: 缺少变量 {e}")

# ✅ 正确提供变量
result = prompt.format(topic="Python")
print(f"成功: {result}")

错误: 缺少变量 'topic'
成功: 介绍 Python


#### 变量名匹配验证

In [9]:
from langchain_core.prompts import PromptTemplate

# 情况1: 模板变量与声明变量一致
try:
    prompt1 = PromptTemplate(
        input_variables=["topic"],
        template="介绍 {topic}"
    )
    result = prompt1.format(topic="AI")
    print(f"✅ 变量匹配: {result}")
except Exception as e:
    print(f"❌ 错误: {e}")

# 情况2: 模板变量与声明变量不一致
try:
    prompt2 = PromptTemplate(
        input_variables=["subject"],  # 声明 subject
        template="介绍 {topic}"       # 模板用 topic
    )
    result = prompt2.format(topic="AI")
    print(f"⚠️ 意外成功: {result}")
except Exception as e:
    print(f"✅ 检测到不匹配: {e}")

✅ 变量匹配: 介绍 AI
⚠️ 意外成功: 介绍 AI


#### 错误处理机制

In [8]:
from langchain_core.prompts import PromptTemplate

def safe_format_template(prompt_template, **kwargs):
    """安全的模板格式化函数"""
    try:
        result = prompt_template.format(**kwargs)
        return f"✅ 成功: {result}"
    except KeyError as e:
        return f"❌ 缺少变量: {e}"
    except Exception as e:
        return f"❌ 其他错误: {type(e).__name__}: {e}"

# 测试不同场景
prompt = PromptTemplate(
    input_variables=["concept", "level"],
    template="向 {level} 解释 {concept}"
)

print(safe_format_template(prompt, concept="机器学习", level="初学者"))
print(safe_format_template(prompt, concept="机器学习"))  # 缺少 level
print(safe_format_template(prompt, level="初学者"))     # 缺少 concept

✅ 成功: 向 初学者 解释 机器学习
❌ 缺少变量: 'level'
❌ 缺少变量: 'concept'


### MessagesPlaceholder

历史消息

In [5]:
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

template = ChatPromptTemplate.from_messages([
    ("system", "你是一个智能助手，能够记住对话历史。"),
    MessagesPlaceholder("history"),
    ("user", "{input}")
])

# 准备历史消息
history_messages = [
    HumanMessage(content="你好，我叫小明"),
    AIMessage(content="你好小明，很高兴认识你！有什么我可以帮助你的吗？")
]

# 格式化模板，包含历史消息和新的用户输入
formatted_messages = template.format_messages(
    history=history_messages,
    input="你还记得我的名字吗？"
)

# 输出格式化后的 ChatPromptTemplate
for i, msg in enumerate(formatted_messages):
    role = msg.__class__.__name__.replace('Message', '')
    print(f"{i+1}. {role}: {msg.content}")

1. System: 你是一个智能助手，能够记住对话历史。
2. Human: 你好，我叫小明
3. AI: 你好小明，很高兴认识你！有什么我可以帮助你的吗？
4. Human: 你还记得我的名字吗？
